## 埋め込みを使用した回帰

回帰とは、カテゴリの中から一つを選ぶのではなく、数値を予測することを意味します。レビューテキストの埋め込みに基づいてスコアを予測します。以下のすべてのタスクについて、データセットを訓練用とテスト用に分割し、未知のデータに対する性能を現実的に評価できるようにします。データセットは[Get_embeddings_from_dataset Notebook](Get_embeddings_from_dataset.ipynb)で作成されます。

レビューのスコアを予測しますが、これは1から5までの数値です（1つ星が否定的、5つ星が肯定的）。

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)

X_train, X_test, y_train, y_test = train_test_split(list(df.embedding.values), df.Score, test_size=0.2, random_state=42)

rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X_train, y_train)
preds = rfr.predict(X_test)

mse = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

print(f"text-embedding-3-small performance on 1k Amazon reviews: mse={mse:.2f}, mae={mae:.2f}")


text-embedding-3-small performance on 1k Amazon reviews: mse=0.65, mae=0.52


In [3]:
bmse = mean_squared_error(y_test, np.repeat(y_test.mean(), len(y_test)))
bmae = mean_absolute_error(y_test, np.repeat(y_test.mean(), len(y_test)))
print(
    f"Dummy mean prediction performance on Amazon reviews: mse={bmse:.2f}, mae={bmae:.2f}"
)


Dummy mean prediction performance on Amazon reviews: mse=1.73, mae=1.03


埋め込みが平均誤差0.53でスコアを予測できることがわかります。これは、レビューの半分を完璧に予測し、残りの半分を1つ星分外すことにほぼ相当します。

分類器を訓練してラベルを予測することもできますし、既存のMLモデル内で埋め込みを使用して自由テキストの特徴量をエンコードすることも可能です。